<h1 align=center><font size = 5>Machine Learning models for aiding the decision-making process in emergency departments</font></h1>

<h1>Tabla comparativa de algoritmos<h1>
<h3>Descripción</h3>
<p>
En este script se desarrollará una serie de predicciones basadas en datos del hospital San Juan de Dios Curicó, correspondientes al año 2018 representados por registros de urgencias. El objetivo es predecir, mediante un conjunto de algoritmos, la necesidad de hospitalización de un paciente de urgencias,tomando como input, datos proporcionados por el paciente en la etapa de registro, sus signos vitales registrados en la etapa de triage y el diagnóstico ofrecido por el médico tratante.
Se correrán algoritmos de prodicción tales como árboles y bosques de desición, regresión logística, support vector machines y redes neuronales. Para finalmente evaluar el rendimiento de cada algoritmo en términos de la predicción, mediante indicadores tales como Acuraccy, F1-Score, Curva ROC, Índice de Jaccard y logloss
</p>

<h1 id="Descripción de datos">Descripción de datos</h1>
<p>
Los datos utilizados fueros proporcionados por el Hospital San Juan de Dios, Curicó, Chile y corresponden a 4.971 registros de pacientes que asistieron a urgencias durante el periodo comprendido entre el 1 de enero de 2018 y agosto de 2019, los datos fueron limpiados y transformados en un script desarrollado previamente
<ul>
    <li>Datos: <a href="https://drive.google.com/open?id=1QmJTDCx060GJw3QHyjl5OhmR4szE8Ojh" target="_blank">https://drive.google.com/open?id=1QmJTDCx060GJw3QHyjl5OhmR4szE8Ojh</a></li>
    <li>Tipo de datos: csv</li>
   </ul>
<p>
Las variables presentes en los datos se describen a continuación:
<ul>    
   
   <li><b>PAC_EDAD</b>: corresponde a la edad del paciente en enteros</li>
   <li><b>MOTIVO_CONSULTA</b>: corresponde a la razón por la que el paciente acude al servicio de urgencias string</li>
   <li><b>MEDIO</b>: corresponde al medio de llegada, mediante el que el paciente acude al servicio de urgencias</li>
   <li><b>SEXO</b>: corresponde al sexo del paciente</li>
   <li><b>CAT</b>: corresponde a la categoría de gravedad asignada al paciente en el proceso de Triage</li>
   <li><b>PRESION_SIST</b>: corresponde la presión sistólica del paciente </li>
   <li><b>PRESION_DIAST</b>: corresponde la presióndiastólica del paciente</li>
   <li><b>SATO2</b>: Dato numérico que representa la saturometria del paciente (Nivel de oxigeno en la sangre)</li>
   <li><b>TEMPERATURA</b>: corresponde a la temperatura corporal del paciente en el momento de la categorización</li>
   <li><b>GLASGOW</b>: corresponde a al nivel registrado por el paciente en la escala Glasgow</li>
   <li><b>DM</b>: corresponde si el paciente presenta o no Diabetes Mellitus</li>
   <li><b>EVA</b>: corresponde si se aplica al paciente una evaluación de vias aéreas</li>
   <li><b>HGT</b>: corresponde a la medida de azucar en la sangre del paciente</li>
   <li><b>LCFA</b>: corresponde a si el paciente presenta obstrucción crónica de vías aéreas</li>
   <li><b>FR</b>: corresponde a la frecuencia respiratoria del paciente</li>
   <li><b>HTA</b>: corresponde a si el paciente posee Hipertención Arterial</li>
   <li><b>HORA_INSC</b>: corresponde a la hora en la que el paciente fue categorizado</li>
   <li><b>MIN_INSC</b>: corresponde al minuto en que el paciente fue categorizado</li>
   <li><b>TIEMPO_ESPERA_CAT</b>: corresponde al tiempo que espera el paciente luego de ser registrado, para ser categorizado</li>
      
</ul>
</p>

Cargar paquetes necesarios

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import sklearn as sk  
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#Métodos de tuneo de parámetros
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Lectura de datos

In [2]:
archivo= 'df_limpia_dias_hosp.csv'
df_urg= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [3]:
df_urg.columns

Index(['Unnamed: 0', 'PAC_EDAD', 'COMUNA', 'PREVISION', 'MOTIVO_CONSULTA',
       'MEDIO', 'DESC_EVENTO', 'SEXO', 'SATO2', 'TEMPERATURA', 'GLASGOW', 'DM',
       'EVA', 'HGT', 'LCFA', 'FR', 'HTA', 'CAT', 'DESTINO', 'VIOLENCIA',
       'PROCEDENCIA', 'DIAGNOSTICO', 'DIAS_HOSP', 'DETALLE_INT',
       'TIEMPO_ESPERA_CAT', 'TIEMPO_ESPERA_ATENCION', 'DURACION_ATENCION',
       'ESPERA_HOSP_EFECTIVA', 'MES_INSC', 'SEMANA_INSC', 'DIA_MES_INSC',
       'DIA_SEMANA_INSC', 'HORA_INSC', 'MIN_INSC', 'DIA_MES_FIN_ATENCION',
       'DIA_SEMANA_FIN_ATENCION', 'HORA_INSC_FIN_ATENCION',
       'MIN_INSC_FIN_ATENCION', 'MES_HOSP', 'SEMANA_HOSP', 'DIA_MES_HOSP',
       'DIA_SEMANA_HOSP', 'PRESION_SIST', 'PRESION_DIAST'],
      dtype='object')

Seleccionar las variables relevantes para la predicción y asignarselas a la matriz X, de variables independientes

In [4]:
df_T_hosp = df_urg[["PAC_EDAD",'MOTIVO_CONSULTA','MEDIO', 'SEXO','DESC_EVENTO','GLASGOW', 'DM', 'EVA','HGT', 'LCFA', 'FR', 
                 'HTA','CAT','DESTINO','DIAS_HOSP','TIEMPO_ESPERA_CAT', 'TIEMPO_ESPERA_ATENCION', 'DURACION_ATENCION',
       'ESPERA_HOSP_EFECTIVA', 'MES_INSC', 'SEMANA_INSC', 'DIA_MES_INSC',
       'DIA_SEMANA_INSC', 'HORA_INSC', 'MIN_INSC', 'DIA_MES_FIN_ATENCION',
       'DIA_SEMANA_FIN_ATENCION', 'HORA_INSC_FIN_ATENCION',
       'MIN_INSC_FIN_ATENCION', 'MES_HOSP', 'SEMANA_HOSP', 'DIA_MES_HOSP',
       'DIA_SEMANA_HOSP', 'PRESION_SIST', 'PRESION_DIAST']]

In [5]:
X = df_T_hosp[["PAC_EDAD",'MOTIVO_CONSULTA','MEDIO', 'SEXO','GLASGOW', 'DM', 'EVA','HGT', 'LCFA', 'FR', 
                 'HTA','CAT','DESTINO','TIEMPO_ESPERA_CAT', 'TIEMPO_ESPERA_ATENCION', 'DURACION_ATENCION',
       'ESPERA_HOSP_EFECTIVA', 'MES_INSC', 'SEMANA_INSC', 'DIA_MES_INSC',
       'DIA_SEMANA_INSC', 'HORA_INSC', 'MIN_INSC', 'DIA_MES_FIN_ATENCION',
       'DIA_SEMANA_FIN_ATENCION', 'HORA_INSC_FIN_ATENCION',
       'MIN_INSC_FIN_ATENCION', 'MES_HOSP', 'SEMANA_HOSP', 'DIA_MES_HOSP',
       'DIA_SEMANA_HOSP', 'PRESION_SIST', 'PRESION_DIAST']].values

In [6]:
X[0:2]

array([[79, 'OTROS, ESPECIFICAR', 'POR SUS PROPIOS MEDIOS', 'FEMENINO',
        15.0, 'S', 3.0, 121.0, 'S', 14.0, 'S', 'C3', 'DOMICILIO', 11, 93,
        268, 46.0, 5, 21, 23, 2, 17, 1, 23, 2, 23, 13, 3.0, 12.0, 22.0,
        3.0, 159.0, 64.0],
       [36, 'OTROS, ESPECIFICAR', 'SAMU MOVIL AVANZADO', 'MASCULINO',
        15.0, 'N', 5.0, 114.0, 'N', 17.0, 'N', 'C3', 'OTRO', 2, 13, 209,
        1315.0, 5, 20, 17, 3, 22, 18, 18, 4, 2, 4, 1.0, 5.0, 30.0, 1.0,
        131.0, 84.0]], dtype=object)

 <h1 id="Preprocesamiento de datos">Preprocesamiento de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben ser numéricos, en este caso siguen siendo en su mayoría categoricos pero serán transformados a nominales.
</p>

In [7]:
from sklearn import preprocessing

Preprocesando motivo de consulta

In [8]:
le_motivo_consulta = preprocessing.LabelEncoder()
le_motivo_consulta.fit(['OTROS, ESPECIFICAR','DOLOR Y/O MOLESTIAS ABDOMEN'
,'PROBLEMAS RESPIRATORIOS','DOLOR Y/O MOLESTIAS TORAX','DOLOR Y/O MOLESTIAS EXTREMIDADES',
'ACCIDENTE/TRAUMATISMO OTRO TIPO','ACCIDENTE/TRAUMATISMO EN EL HOGAR', 
'DIABETES MELLITUS DESCOMPENSADA','VOMITOS',
'DOLOR Y/O MOLESTIAS CABEZA Y CUELLO','PRESION ARTERIAL ELEVADA',
'ACCIDENTE/TRAUMATISMO VIA PUBLICA','FIEBRE','CONVULSIONES',
'CONSTATACION DE LESIONES',
'ALCOHOLEMIA + CONST.LESIONES','DIARREA','AGRESION Y/O VIOLENCIA',
'MORDEDURA ANIMAL','MORDEDURA ARAÃA','QUEMADURAS','PROCEDIMIENTO DE HEMODINAMIA',
'ALCOHOLEMIA','HERIDAS Y/O HEMORRAGIA' ])

X[:,1] = le_motivo_consulta.transform(X[:,1]) 

In [9]:
X[0:]

array([[79, 18, 'POR SUS PROPIOS MEDIOS', ..., 3.0, 159.0, 64.0],
       [36, 18, 'SAMU MOVIL AVANZADO', ..., 1.0, 131.0, 84.0],
       [37, 18, 'POR SUS PROPIOS MEDIOS', ..., 4.0, 133.0, 64.0],
       ...,
       [83, 18, 'AMBULANCIA APS', ..., 1.0, 141.0, 61.0],
       [41, 1, 'POR SUS PROPIOS MEDIOS', ..., 6.0, 136.0, 81.0],
       [82, 18, 'AMBULANCIA HOSPITAL', ..., 3.0, 115.0, 56.0]],
      dtype=object)

Preprocesando medio de llegada

In [10]:
le_medio = preprocessing.LabelEncoder()
le_medio.fit([ 'POR SUS PROPIOS MEDIOS','SAMU MOVIL AVANZADO',
'AMBULANCIA APS','AMBULANCIA HOSPITAL','SAMU MOVIL BASICO',
'VEHICULO POLICIAL CARABINEROS','OTRO MEDIO TERRESTRE PAGADO',
'GENDARMERIA','AMBULANCIA CLINICAS PRIVADAS'])
X[:,2] = le_medio.transform(X[:,2])

Preprocesamiento Sexo

In [11]:
le_sexo = preprocessing.LabelEncoder()
le_sexo.fit([ 'FEMENINO','MASCULINO'])
X[:,3] = le_sexo.transform(X[:,3])

Preprocesamiento DM (Diabetes Mellitus)

In [12]:
le_DM = preprocessing.LabelEncoder()
le_DM.fit([ 'S', 'N','D'])
X[:,5] = le_DM.transform(X[:,5])

Preprocesamiento LCFA (Limintación crónica del flujo aéreo)

In [13]:
le_LCFA = preprocessing.LabelEncoder()
le_LCFA.fit([ 'S', 'N','D'])
X[:,8] = le_LCFA.transform(X[:,8]) 

Preprocesamiento HTA (Hipertención Arterial)

In [14]:
le_HTA = preprocessing.LabelEncoder()
le_HTA.fit([ 'S', 'N','D'])
X[:,10] = le_HTA.transform(X[:,10]) 

Preprocesamiento CAT ( Categoría)

In [15]:
le_CAT = preprocessing.LabelEncoder()
le_CAT.fit([ 'C1', 'C2','C3','C4','C5'])
X[:,11] = le_CAT.transform(X[:,11]) 

Preprocesamiento DESTINO (Hospitalizar o no)

In [16]:
le_DESTINO= preprocessing.LabelEncoder()
le_DESTINO.fit(['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO','A.P.S.'])
X[:,12] = le_DESTINO.transform(X[:,12]) 

Asignación de la variable dependiente a predecir (categoría), al vector y

In [17]:
y = df_T_hosp["DIAS_HOSP"]

 <h1 id="Normalización de datos">Normalización de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben estar en una escala similar, es por ello que optamos por normalizarlos para que los valores estén en un rango entre -2 y 2. Esta medida no altera los resultados aunque si faborece a la eficiencia de los algoritmos
</p>

In [18]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 0.92243813,  0.58010589,  0.20755605, -0.8660254 ,  0.26352314,
         0.75592895, -0.82911071, -0.4491311 ,  4.96655481, -1.12386049,
         0.77988964,  0.75592895, -0.15673224, -0.06931612,  1.41401212,
         0.58282432, -1.30025282,  0.79296435,  0.95953793,  0.88861454,
        -0.61744679,  0.57781265, -1.73655342,  0.92576862, -0.69708118,
         1.33170513, -0.80772554, -0.80255943, -0.74627996,  0.73138838,
         0.18058168,  0.44807237, -1.10706919],
       [-1.46738398,  0.58010589,  0.70698778,  1.15470054,  0.26352314,
        -1.32287566,  0.11668965, -0.54662674, -0.20134682, -0.0238106 ,
        -1.15725559,  0.75592895,  1.85466483, -0.12472104, -0.41319884,
         0.24055817,  1.62396933,  0.79296435,  0.82039589,  0.1868369 ,
        -0.0514539 ,  1.44340748, -0.71215124,  0.33566776,  0.42115322,
        -1.57040699, -1.32988394, -1.42053019, -1.24104612,  1.60962471,
        -0.88901752, -0.57636457, -0.02528838],
       [-1.41180672,  0.5801

 <h1 id="Análisis de componentes principales">Análisis de componentes principales</h1>
<p>
Se realizó el análisis de componentes principales con el fin de reducir la dimensionalidad de la base de datos utilizada para la predicción de categoría. El objetivo de reducir la dimensionalidad de la base de datos es agilizar los procesos de entrenamiento y predicción de la categoria de pacientes, además de identificar las variables que presentan una mayor utilidad para esta, descartando las que no aportan indormación a la predicción.
</p>

In [23]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X)
pca.explained_variance_ratio_
X.shape
#pca.n_components_ 

17

 <h1 id="Configurando algoritmos">Configurando algoritmos</h1>
<p>
En esta sección se definen parámetros necesarios para la correcta aplicación de los algoritmos a implementar, además de seccionar el conjunto de datos en datos de prueba(30%)y de entrenamiento (70%). Los parámetros escogidos pueden ser modificados con el fin de obtener resultados diferentes
</p>

Carga de paquetes

In [19]:
from sklearn.model_selection import train_test_split

Segmentación del conjunto de datos

In [20]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

Árbol de desición

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
DT= DecisionTreeRegressor(random_state=0)
DT.fit(X_trainset,y_trainset)
yhat_1 = DT.predict(X_testset)
DT_CV = cross_val_score(DT, X, y, cv=10)
DT_R2=round(DT.score(X_testset, y_testset),4)
DT_MAX=max_error( y_testset,yhat_1)
DT_MAD=mean_absolute_error( y_testset,yhat_1)
DT_MSE =round(mean_squared_error(y_testset,yhat_1),4)
DT_r22= round(r2_score(y_testset,yhat_1),4)
DT_r22
yhat_1[0:10]

array([21., 20.,  3.,  3., 12., 10.,  3., 19., 32., 32.])

In [22]:
y_testset[0:10]

31     8.0
32     0.0
23     4.0
60    29.0
8     36.0
6     21.0
52     3.0
9     36.0
12     4.0
27     5.0
Name: DIAS_HOSP, dtype: float64

In [30]:
DT_parameters = [{'criterion': ['mse', 'friedman_mse', 'mae'], 'max_depth': np.arange(1, 21)},{'min_samples_leaf': [1, 5, 10, 20, 50, 100]}]
DT_GS = GridSearchCV(DecisionTreeRegressor(random_state=42), DT_parameters, verbose=1, cv=3)
DT_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 66 candidates, totalling 198 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 198 out of 198 | elapsed:    0.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=42, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['mse', 'friedman_mse', 'mae'],
                          'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20

In [31]:
DT_GS.best_estimator_

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=20, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [32]:
yhat_1_GS= DT_GS.predict(X_testset)
DT_R2_GS=round(DT_GS.score(X_testset, y_testset),4)
DT_MAX_GS=max_error( y_testset,yhat_1_GS)
DT_MAD_GS=mean_absolute_error( y_testset,yhat_1_GS)
DT_MSE_GS =round(mean_squared_error(y_testset,yhat_1_GS),4)
DT_r22_GS= round(r2_score(y_testset,yhat_1_GS),4)
DT_r22_GS

-0.1483

In [33]:
resultados_DT = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Árboles de decisión':[DT_R2,DT_MAX,DT_MAD,DT_MSE],
             'Grid Search':[DT_R2_GS,DT_MAX_GS,DT_MAD_GS,DT_MSE_GS]}
Tabla_resultados_DT=pd.DataFrame(resultados_DT)
print(Tabla_resultados_DT)

  índices de rendimiento  Árboles de decisión  Grid Search
0                    R^2              -1.9545    -0.148300
1              MAX error              63.0000    20.437500
2                    MAD              14.6250    11.279018
3                    MSE             388.8750   151.144100


Bosque de desición

In [34]:
from sklearn.ensemble import RandomForestRegressor
RF= RandomForestRegressor(max_depth=2, random_state=0)
RF.fit(X_trainset,y_trainset)
yhat_2 = RF.predict(X_testset)
RF_R2=round(RF.score(X_testset, y_testset),4)
RF_MAX=round(max_error( y_testset,yhat_2),4)
RF_MAD=round(mean_absolute_error( y_testset,yhat_2),4)
RF_MSE =round(mean_squared_error(y_testset,yhat_2),4)
RF_MSE

197.9246

In [35]:
RF_parameters ={ 'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [2,3,4,5,6,7,8],'criterion' :['mse', 'friedman_mse', 'mae']}
RF_GS = GridSearchCV(estimator=RF,param_grid=RF_parameters, cv= 3)
RF_GS.fit(X_trainset,y_trainset)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=2,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=0,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=Non

In [23]:
RF_GS.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [37]:
yhat_2_GS= RF_GS.predict(X_testset)
RF_R2_GS=round(RF.score(X_testset, y_testset),4)
RF_MAX_GS=round(max_error( y_testset,yhat_2_GS),4)
RF_MAD_GS=round(mean_absolute_error( y_testset,yhat_2_GS),4)
RF_MSE_GS =round(mean_squared_error(y_testset,yhat_2_GS),4)
RF_MSE_GS

160.9803

In [40]:
resultados_RF = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Bosques de decisión':[RF_R2,RF_MAX,RF_MAD,RF_MSE],
             'Grid Search':[RF_R2_GS,RF_MAX_GS,RF_MAD_GS,RF_MSE_GS]}
Tabla_resultados_RF=pd.DataFrame(resultados_RF)
print(Tabla_resultados_RF)

  índices de rendimiento  Bosques de decisión  Grid Search
0                    R^2              -0.5037      -0.5037
1              MAX error              27.1926      27.0125
2                    MAD              12.1118      10.7566
3                    MSE             197.9246     160.9803


Regresión logística (multi-class)

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='newton-cg',penalty='l2').fit(X_trainset,y_trainset)
yhat_3= LR.predict(X_testset)
LR_R2=round(LR.score(X_testset, y_testset),4)
LR_MAX=round(max_error( y_testset,yhat_3),4)
LR_MAD=round(mean_absolute_error( y_testset,yhat_3),4)
LR_MSE =round(mean_squared_error(y_testset,yhat_3),4)
LR_MSE

213.375

In [43]:
LR_parameters ={'C':np.logspace(0.1,3,7), 'penalty':['l1', 'l2', 'elasticnet'],}
LR_GS = GridSearchCV(estimator=LR,param_grid=LR_parameters, cv= 3,verbose=0,)
LR_GS.fit(X_trainset,y_trainset)

C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

  FitFailedWarning)
C:\Users\jmoya\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit fai

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=0.01, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='newton-cg',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([   1.25892541,    3.83118685,   11.65914401,   35.48133892,
        107.97751623,  328.59932476, 1000.        ]),
                         'penalty': ['l1', 'l2', 'elasticnet']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [44]:
LR_GS.best_estimator_

LogisticRegression(C=1.2589254117941673, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
yhat_3_GS= LR_GS.predict(X_testset)
LR_R2_GS=round(LR.score(X_testset, y_testset),4)
LR_MAX_GS=round(max_error( y_testset,yhat_3_GS),4)
LR_MAD_GS=round(mean_absolute_error( y_testset,yhat_3_GS),4)
LR_MSE_GS =round(mean_squared_error(y_testset,yhat_3_GS),4)
LR_MSE_GS

180.8333

In [47]:
resultados_LR = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Regresión Logística':[LR_R2,LR_MAX,LR_MAD,LR_MSE],
             'Grid Search':[LR_R2_GS,LR_MAX_GS,LR_MAD_GS,LR_MSE_GS]}
Tabla_resultados_LR=pd.DataFrame(resultados_LR)
print(Tabla_resultados_LR)

  índices de rendimiento  Regresión Logística  Grid Search
0                    R^2               0.1667       0.1667
1              MAX error              33.0000      30.0000
2                    MAD               9.5417       9.7500
3                    MSE             213.3750     180.8333


Regresión Lasso

In [48]:
from sklearn import linear_model
LssR = linear_model.Lasso(alpha=0.1).fit(X_trainset,y_trainset)
yhat_6= LssR.predict(X_testset)
LssR_R2=round(LssR.score(X_testset, y_testset),4)
LssR_MAX=round(max_error( y_testset,yhat_6),4)
LssR_MAD=round(mean_absolute_error( y_testset,yhat_6),4)
LssR_MSE =round(mean_squared_error(y_testset,yhat_6),4)
LssR_MSE

286.5444

In [50]:
LssR_parameters ={'alpha':[0.005, 0.02, 0.03, 0.05, 0.06]}
LssR_GS = GridSearchCV(estimator=LssR,param_grid=LssR_parameters, cv= 3, verbose=True, n_jobs=-1)
LssR_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    2.6s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Lasso(alpha=0.1, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.005, 0.02, 0.03, 0.05, 0.06]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [51]:
LssR_GS.best_estimator_

Lasso(alpha=0.06, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [55]:
yhat_6_GS= LssR_GS.predict(X_testset)
LssR_R2_GS=round(LssR.score(X_testset, y_testset),4)
LssR_MAX_GS=round(max_error( y_testset,yhat_6_GS),4)
LssR_MAD_GS=round(mean_absolute_error( y_testset,yhat_6_GS),4)
LssR_MSE_GS =round(mean_squared_error(y_testset,yhat_6_GS),4)
LssR_MSE_GS

296.473

In [57]:
resultados_LssR = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Regresión Lasso':[LssR_R2,LssR_MAX,LssR_MAD,LssR_MSE],
             'Grid Search':[LssR_R2_GS,LssR_MAX_GS,LssR_MAD_GS,LssR_MSE_GS]}
Tabla_resultados_LssR=pd.DataFrame(resultados_LssR)
print(Tabla_resultados_LssR)

  índices de rendimiento  Regresión Lasso  Grid Search
0                    R^2          -1.1770      -1.1770
1              MAX error          31.1565      29.9883
2                    MAD          15.1231      15.3464
3                    MSE         286.5444     296.4730


Redes Neuronales

In [58]:
from sklearn.neural_network import MLPRegressor
NN = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_trainset,y_trainset)
yhat_4 = NN.predict(X_testset)
NN_R2=round(NN.score(X_testset, y_testset),4)
NN_MAX=round(max_error( y_testset,yhat_4),4)
NN_MAD=round(mean_absolute_error( y_testset,yhat_4),4)
NN_MSE =round(mean_squared_error(y_testset,yhat_4),4)
NN_MSE

179.695

In [59]:
NN_parameters ={'solver': ['lbfgs'], 'max_iter': [500,1000,1500], 'alpha': 10.0 ** -np.arange(1, 7), 'hidden_layer_sizes':np.arange(5, 12), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
NN_GS = GridSearchCV(estimator=NN,param_grid=NN_parameters, cv= 3, verbose=True, n_jobs=-1)
NN_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 1260 candidates, totalling 3780 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2244 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 3780 out of 3780 | elapsed:  2.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=1e-05,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(5, 2),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=200, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=1,...
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06]),
                         '

In [60]:
NN_GS.best_estimator_

MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=6, learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=5, shuffle=True, solver='lbfgs',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [61]:
yhat_4_GS= NN_GS.predict(X_testset)
NN_R2_GS=round(NN.score(X_testset, y_testset),4)
NN_MAX_GS=round(max_error( y_testset,yhat_4_GS),4)
NN_MAD_GS=round(mean_absolute_error( y_testset,yhat_4_GS),4)
NN_MSE_GS =round(mean_squared_error(y_testset,yhat_4_GS),4)
NN_MSE_GS

284.9929

In [62]:
resultados_NN = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Red Neuronal':[NN_R2,NN_MAX,NN_MAD,NN_MSE],
             'Grid Search':[NN_R2_GS,NN_MAX_GS,NN_MAD_GS,NN_MSE_GS]}
Tabla_resultados_NN=pd.DataFrame(resultados_NN)
print(Tabla_resultados_NN)

  índices de rendimiento  Red Neuronal  Grid Search
0                    R^2       -0.3652      -0.3652
1              MAX error       29.3941      55.7347
2                    MAD       10.1660      11.9096
3                    MSE      179.6950     284.9929


Support Vector Machine

In [63]:
from sklearn.svm import LinearSVR
SVM = LinearSVR(random_state=0, tol=1e-5)
SVM.fit(X_trainset, y_trainset) 
yhat_5 = SVM.predict(X_testset)
SVM_R2=round(SVM.score(X_testset, y_testset),4)
SVM_MAX=round(max_error( y_testset,yhat_5),4)
SVM_MAD=round(mean_absolute_error( y_testset,yhat_5),4)
SVM_MSE =round(mean_squared_error(y_testset,yhat_5),4)
SVM_MSE

151.8974

In [72]:
SVM_parameters ={'C': [0.1,1, 10, 100]}
SVM_GS = GridSearchCV(estimator=SVM,param_grid=SVM_parameters, cv= 3, verbose=True, n_jobs=-1)
SVM_GS.fit(X_trainset,y_trainset)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    2.6s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0,
                                 fit_intercept=True, intercept_scaling=1.0,
                                 loss='epsilon_insensitive', max_iter=1000,
                                 random_state=0, tol=1e-05, verbose=0),
             iid='deprecated', n_jobs=-1, param_grid={'C': [0.1, 1, 10, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [73]:
SVM_GS.best_estimator_

LinearSVR(C=1, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=0, tol=1e-05, verbose=0)

In [74]:
yhat_5_GS= SVM_GS.predict(X_testset)
SVM_R2_GS=round(SVM.score(X_testset, y_testset),4)
SVM_MAX_GS=round(max_error( y_testset,yhat_5_GS),4)
SVM_MAD_GS=round(mean_absolute_error( y_testset,yhat_5_GS),4)
SVM_MSE_GS =round(mean_squared_error(y_testset,yhat_5_GS),4)
SVM_MSE_GS

151.8974

In [75]:
resultados_SVM = {'índices de rendimiento':['R^2','MAX error','MAD','MSE'],
             'Red Neuronal':[SVM_R2,SVM_MAX,SVM_MAD,SVM_MSE],
             'Grid Search':[SVM_R2_GS,SVM_MAX_GS,SVM_MAD_GS,SVM_MSE_GS]}
Tabla_resultados_SVM=pd.DataFrame(resultados_SVM)
print(Tabla_resultados_SVM)

  índices de rendimiento  Red Neuronal  Grid Search
0                    R^2       -0.1540      -0.1540
1              MAX error       26.1806      26.1806
2                    MAD        9.9512       9.9512
3                    MSE      151.8974     151.8974


In [76]:
resultados = {'Algoritmos de regresión':['Árboles de decisión','Bosques de decisión','Regresión Logística','Regresión Lasso','Red Neuronal','Support Vector Machine'],
             'R^2':[DT_R2,RF_R2,LR_R2,LssR_R2,NN_R2,SVM_R2],
             'Max error':[DT_MAX,RF_MAX,LR_MAX,LssR_MAX,NN_MAX,SVM_MAX],
             'MAD':[DT_MAD,RF_MAD,LR_MAD,LssR_MAD,NN_MAD,SVM_MAD],
             'MSE':[DT_MSE,RF_MSE,LR_MSE,LssR_MSE,NN_MSE,SVM_MSE]}
Tabla_resultados=pd.DataFrame(resultados)
print(Tabla_resultados)

  Algoritmos de regresión     R^2  Max error      MAD       MSE
0     Árboles de decisión -1.9545    63.0000  14.6250  388.8750
1     Bosques de decisión -0.5037    27.1926  12.1118  197.9246
2     Regresión Logística  0.1667    33.0000   9.5417  213.3750
3         Regresión Lasso -1.1770    31.1565  15.1231  286.5444
4            Red Neuronal -0.3652    29.3941  10.1660  179.6950
5  Support Vector Machine -0.1540    26.1806   9.9512  151.8974
